In [ ]:
#Use Xavier / He after this

import tensorflow
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten


def get_global_configuration():
    """ Retrieve configuration of the training process. """
    num_layers_to_add = 10
    return num_layers_to_add


def get_model_configuration():
    """ Retrieve configuration for the model. """
    img_width, img_height = 32, 32
    no_classes = 10
    batch_size = 250
    no_epochs = 25
    validation_split = 0.2
    verbosity = 0
    return img_width, img_height, no_classes, batch_size,\
        no_epochs, validation_split, verbosity


def get_dataset():
    """ Load and convert dataset into inputs and targets """
    # Load relevant model configuration
    img_width, img_height, no_classes, _, _, _, _ = get_model_configuration()

    # Load cifar10 dataset
    (input_train, target_train), (input_test, target_test) = cifar10.load_data()

    # Reshape data
    input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 3)
    input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 3)
    input_shape = (img_width, img_height, 3)

    # Parse numbers as floats
    input_train = input_train.astype('float32')
    input_test = input_test.astype('float32')

    # Convert into [0, 1] range.
    input_train = input_train / 255
    input_test = input_test / 255

    # Convert target vectors to categorical targets
    target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
    target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

    # Return data
    return input_train, input_test, target_train, target_test


def create_base_model():
    """ Create a base model: one Dense layer. """
    # Retrieve relevant model configuration
    _, _, no_classes, _, _, _, _ = get_model_configuration()
    # Create model instance and add initial layers
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))
    # Return model
    return model


def add_extra_layer(model):
    """ Add an extra Dense layer to the model. """
    # Define the layer that must be added
    layer_to_add = Dense(256, activation='relu')
    # Temporarily assign the output layer to a variable
    output_layer = model.layers[-1]
    # Set all upstream layers to nontrainable
    for layer in model.layers:
        layer.trainable = False
    # Remove output layer and add new layer
    model.pop()
    model.add(layer_to_add)
    # Re-add output layer
    model.add(output_layer)
    # Return trained model, with extra layer
    return model


def train_model(model, data):
    """ Compile and train a model. """
    # Retrieve relevant model configuration
    _, _, _, batch_size, no_epochs, validation_split, verbosity = get_model_configuration()
    # Decompose data into components
    input_train, input_test, target_train, target_test = data
    # Compile  model
    model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                  optimizer=tensorflow.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    # Train model
    model.fit(input_train, target_train,
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              validation_split=validation_split)
    # Evaluate model
    score = model.evaluate(input_test, target_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    # Return trained model
    return model


def training_process():
    """ Run the training process. """
    # Create the base model
    model = create_base_model()
    # Get data
    data = get_dataset()
    # Apply greedy layer-wise training
    num_layers_to_add = get_global_configuration()
    for i in range(num_layers_to_add):
        # Train and evaluate current model
        model = train_model(model, data)
        # Add extra layer
        model = add_extra_layer(model)


if __name__ == "__main__":
    training_process()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Test loss: 1.5009902715682983 / Test accuracy: 0.4675999879837036
Test loss: 1.4323339462280273 / Test accuracy: 0.5
Test loss: 1.45620858669281 / Test accuracy: 0.5029000043869019
Test loss: 1.5449928045272827 / Test accuracy: 0.49799999594688416
Test loss: 1.699157476425171 / Test accuracy: 0.48559999465942383
Test loss: 1.8477650880813599 / Test accuracy: 0.48010000586509705
Test loss: 1.9427380561828613 / Test accuracy: 0.47609999775886536
Test loss: 2.0550317764282227 / Test accuracy: 0.47540000081062317
Test loss: 2.1350057125091553 / Test accuracy: 0.47429999709129333
Test loss: 2.180410385131836 / Test accuracy: 0.4697999954223633


In [ ]:
import tensorflow
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.initializers import HeNormal


def get_global_configuration():
    """ Retrieve configuration of the training process. """
    num_layers_to_add = 10
    return num_layers_to_add


def get_model_configuration():
    """ Retrieve configuration for the model. """
    img_width, img_height = 32, 32
    no_classes = 10
    batch_size = 250
    no_epochs = 25
    validation_split = 0.2
    verbosity = 0
    return img_width, img_height, no_classes, batch_size, no_epochs, validation_split, verbosity


def get_dataset():
    """ Load and preprocess the CIFAR-10 dataset. """
    img_width, img_height, no_classes, _, _, _, _ = get_model_configuration()

    # Load CIFAR-10 dataset
    (input_train, target_train), (input_test, target_test) = cifar10.load_data()

    # Normalize pixel values to [0, 1] range
    input_train = input_train.astype('float32') / 255
    input_test = input_test.astype('float32') / 255

    # Convert target labels to one-hot encoding
    target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
    target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

    return input_train, input_test, target_train, target_test


def create_base_model():
    """ Create a base model with He initialization. """
    _, _, no_classes, _, _, _, _ = get_model_configuration()

    model = Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer=HeNormal()))
    model.add(Dense(no_classes, activation='softmax', kernel_initializer=HeNormal()))

    return model


def add_extra_layer(model):
    """ Add an extra Dense layer with He initialization. """
    layer_to_add = Dense(256, activation='relu', kernel_initializer=HeNormal())

    output_layer = model.layers[-1]  # Store the output layer
    for layer in model.layers:
        layer.trainable = False  # Freeze previous layers

    model.pop()  # Remove the last layer
    model.add(layer_to_add)
    model.add(output_layer)  # Re-add the output layer

    return model


def train_model(model, data):
    """ Compile and train the model. """
    _, _, _, batch_size, no_epochs, validation_split, verbosity = get_model_configuration()
    input_train, input_test, target_train, target_test = data

    # Compile the model with categorical crossentropy and Adam optimizer
    model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                  optimizer=tensorflow.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    # Train the model
    model.fit(input_train, target_train,
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              validation_split=validation_split)

    # Evaluate model performance
    score = model.evaluate(input_test, target_test, verbose=0)
    print(f'Test loss: {score[0]:.4f} / Test accuracy: {score[1]:.4f}')

    return model


def training_process():
    """ Execute the training process with layer-wise training. """
    model = create_base_model()
    data = get_dataset()
    num_layers_to_add = get_global_configuration()

    for i in range(num_layers_to_add):
        print(f'Iteration {i+1}/{num_layers_to_add}: Training model...')
        model = train_model(model, data)
        model = add_extra_layer(model)


if __name__ == "__main__":
    training_process()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Iteration 1/10: Training model...
Test loss: 1.4603 / Test accuracy: 0.4844
Iteration 2/10: Training model...
Test loss: 1.3868 / Test accuracy: 0.5123
Iteration 3/10: Training model...
Test loss: 1.4268 / Test accuracy: 0.5147
Iteration 4/10: Training model...
Test loss: 1.5509 / Test accuracy: 0.5094
Iteration 5/10: Training model...
Test loss: 1.8301 / Test accuracy: 0.4823
Iteration 6/10: Training model...
Test loss: 2.1106 / Test accuracy: 0.4878
Iteration 7/10: Training model...
Test loss: 2.3379 / Test accuracy: 0.4821
Iteration 8/10: Training model...
Test loss: 2.5438 / Test accuracy: 0.4813
Iteration 9/10: Training model...
Test loss: 2.6722 / Test accuracy: 0.4761
Iteration 10/10: Training model...
Test loss: 2.7429 / Test accuracy: 0.4743
